# scRNA-seq QC and Exploratory Data Analysis

**Dataset:** ENCODE short-read shallow (ENCFF914DEE) - 9,000 cell Split-seq C2C12  
**Purpose:** Quality control filtering, normalization, dimensionality reduction, and visualization

This notebook explores the ENCODE short-read single-cell dataset, applies QC filters, normalizes counts, and generates exploratory plots to understand cell quality and gene expression patterns.

## Setup and Data Loading

In [ ]:
import sys
import os
import scanpy as sc
import pandas as pd
import anndata
from anndata import AnnData
import matplotlib.pyplot as plt

# Set plotting parameters
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
# Load data
fname = '../outputs/anndata/short_shallow.h5ad'
short_read_shallow = sc.read(fname)
print(f"Loaded {short_read_shallow.n_obs} cells × {short_read_shallow.n_vars} genes")

## Initial Data Exploration

In [ ]:
# Inspect AnnData structure
print("Observations (cells):")
print(short_read_shallow.obs.head())
print("\nVariables (genes):")
print(short_read_shallow.var.head())
print(f"\nCount matrix shape: {short_read_shallow.X.shape}")

In [ ]:
# Check initial cell and gene identifiers
print("Sample obs_names:")
print(short_read_shallow.obs_names[:5])
print("\nSample var_names:")
print(short_read_shallow.var_names[:5])
print("\nUnnamed column values:")
print(short_read_shallow.obs['Unnamed: 0'][:5])

## Standardize Cell and Gene Names

In [ ]:
# Convert to DataFrame to inspect (WARNING: this densifies the sparse matrix - use only for small previews)
df_preview = pd.DataFrame(
    short_read_shallow.X[:10, :10].todense(),
    index=short_read_shallow.obs['Unnamed: 0'][:10],
    columns=short_read_shallow.var.x[:10]
)
print("Count matrix preview (first 10×10):")
print(df_preview)

In [ ]:
# Standardize obs_names (cell barcodes)
print("Original obs_names:")
print(short_read_shallow.obs_names[:5])

short_read_shallow.obs_names = short_read_shallow.obs['Unnamed: 0'].values
del short_read_shallow.obs['Unnamed: 0']

print("\nUpdated obs_names:")
print(short_read_shallow.obs_names[:5])

# Check uniqueness
are_obs_names_unique = short_read_shallow.obs_names.is_unique
print(f"\nAre obs names unique? {are_obs_names_unique}")

In [ ]:
# Standardize var_names (gene IDs)
short_read_shallow.var_names = short_read_shallow.var['x'].values
short_read_shallow.var.rename({'x': 'transcript_id'}, axis=1, inplace=True)

print("Updated var_names (gene IDs):")
print(short_read_shallow.var_names[:5])
print("\nVariable meta")
print(short_read_shallow.var.head())

are_var_names_unique = short_read_shallow.var_names.is_unique
print(f"\nAre var names unique? {are_var_names_unique}")

## Identify Highly Expressed Genes

In [ ]:
# Plot top expressed genes
sc.pl.highest_expr_genes(short_read_shallow, n_top=20)

## Calculate QC Metrics

In [ ]:
# Compute QC metrics
qc = sc.pp.calculate_qc_metrics(short_read_shallow)
cell_qc_dataframe = qc[0]
gene_qc_dataframe = qc[1]

print('Cell QC metrics:')
print(cell_qc_dataframe.head(5))

print('\nGene QC metrics:')
print(gene_qc_dataframe.head(5))

## Cell QC: Library Size (Total Counts)

In [ ]:
# Plot library size distribution
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(cell_qc_dataframe['total_counts'], bins=1000)
plt.xlabel('Total counts')
plt.ylabel('N cells')
plt.xlim(0, 50000)
plt.ylim(0, 1250)
plt.title('Library size (total counts) per cell')

# Zoom in on low-count cells
plt.subplot(1, 2, 2)
read_count_cutoff = 1000
plt.hist(cell_qc_dataframe['total_counts'], bins=1000)
plt.xlabel('Total counts')
plt.ylabel('N cells')
plt.xlim(0, 20000)
plt.ylim(0, 1250)
plt.axvline(read_count_cutoff, color='red', linestyle='--', label=f'Cutoff: {read_count_cutoff}')
plt.title('Library size with cutoff')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Filter cells by minimum counts
print('Before filtering:')
print(short_read_shallow)

sc.pp.filter_cells(short_read_shallow, min_counts=1000)

print('\nAfter filtering (min_counts=1000):')
print(short_read_shallow)

## Cell QC: Detected Genes

In [ ]:
# Plot detected genes distribution
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(cell_qc_dataframe['n_genes_by_counts'], bins=100)
plt.xlabel('N genes')
plt.ylabel('N cells')
plt.title('Genes detected per cell')

plt.subplot(1, 2, 2)
gene_counts_cutoff = 750
plt.hist(cell_qc_dataframe['n_genes_by_counts'], bins=100)
plt.xlabel('N genes')
plt.ylabel('N cells')
plt.xlim(0, 10000)
plt.ylim(0, 2000)
plt.axvline(gene_counts_cutoff, color='red', linestyle='--', label=f'Cutoff: {gene_counts_cutoff}')
plt.title('Genes detected with cutoff')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Filter cells by minimum genes
print('Before filtering:')
print(short_read_shallow)

sc.pp.filter_cells(short_read_shallow, min_genes=750)

print('\nAfter filtering (min_genes=750):')
print(short_read_shallow)

## Gene QC: Remove Low-Detection Genes

In [ ]:
# Filter genes by minimum cells and counts
print('Before gene filtering:')
print(short_read_shallow)

sc.pp.filter_genes(short_read_shallow, min_cells=2)
sc.pp.filter_genes(short_read_shallow, min_counts=5)

print('\nAfter gene filtering (min_cells=2, min_counts=5):')
print(short_read_shallow)

## Mitochondrial Gene QC

In [ ]:
# Identify mitochondrial transcripts
mt_transcripts = short_read_shallow.var_names.str.contains('mt-')
print(f'Number of MT transcripts: {mt_transcripts.sum()}')
print('\nMitochondrial transcripts:')
print(short_read_shallow.var_names[mt_transcripts])

# Add mitochondrial flag
short_read_shallow.var['mt'] = mt_transcripts

# Calculate QC metrics including mitochondrial percentage
sc.pp.calculate_qc_metrics(
    short_read_shallow,
    qc_vars=['mt'],
    percent_top=None,
    log1p=False,
    inplace=True
)

In [ ]:
# Violin plots of QC metrics
sc.pl.violin(
    short_read_shallow,
    ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
    jitter=0.4,
    multi_panel=True
)

In [ ]:
# Scatter plots: mitochondrial content vs total counts and gene counts
sc.pl.scatter(short_read_shallow, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(short_read_shallow, x='total_counts', y='n_genes_by_counts')

In [ ]:
# Filter cells by gene count and mitochondrial percentage
print('Before final filtering:')
print(short_read_shallow)

short_read_shallow = short_read_shallow[short_read_shallow.obs.n_genes_by_counts < 200000, :]
print('\nAfter removing cells with too many genes:')
print(short_read_shallow)

short_read_shallow = short_read_shallow[short_read_shallow.obs.pct_counts_mt < 20, :]
print('\nAfter removing cells with high mitochondrial content:')
print(short_read_shallow)

## Save QC'd Dataset

In [ ]:
# Save filtered dataset
os.makedirs('../outputs/anndata/qc', exist_ok=True)
short_read_shallow.write('../outputs/anndata/qc/short_shallow_qc.h5ad')
print(f"Saved QC'd dataset: {short_read_shallow.n_obs} cells × {short_read_shallow.n_vars} genes")

## Normalization and Log Transformation

In [ ]:
# Reload QC'd data
short_read_shallow = sc.read('../outputs/anndata/qc/short_shallow_qc.h5ad')

# Check raw count values
print("Raw counts (first 20×20):")
print(short_read_shallow.X[:20, :20].todense())

In [ ]:
# Normalize total counts per cell
sc.pp.normalize_total(short_read_shallow, target_sum=1e4)

print("\nNormalized counts (first 20×20):")
print(short_read_shallow.X[:20, :20].todense())

In [ ]:
# Log1p transform
sc.pp.log1p(short_read_shallow)

print("\nLog-transformed counts (first 20×20):")
print(short_read_shallow.X[:20, :20].todense())

## Highly Variable Genes

In [ ]:
# Identify highly variable genes
sc.pp.highly_variable_genes(
    short_read_shallow,
    min_mean=0.0125,
    max_mean=3,
    min_disp=0.5
)
sc.pl.highly_variable_genes(short_read_shallow)

In [ ]:
# Freeze the current state (saves raw counts)
short_read_shallow.raw = short_read_shallow
print(short_read_shallow)

In [ ]:
# Save normalized and transformed dataset
short_read_shallow.write('../outputs/anndata/qc/short_shallow_norm_log_transformed.h5ad')
print("Saved normalized dataset")

## Dimensionality Reduction: PCA

In [ ]:
# Reload normalized data
short_read_shallow = sc.read('../outputs/anndata/qc/short_shallow_norm_log_transformed.h5ad')

# Run PCA
sc.tl.pca(short_read_shallow, svd_solver='arpack')

# Plot first two principal components
sc.pl.pca(short_read_shallow, annotate_var_explained=True)

In [ ]:
# Plot variance explained by each PC
sc.pl.pca_variance_ratio(short_read_shallow, log=True, n_pcs=50)

## Dimensionality Reduction: t-SNE

In [ ]:
# Run t-SNE
sc.tl.tsne(
    short_read_shallow,
    perplexity=20,
    learning_rate=1000,
    random_state=42,
    n_pcs=40
)

# Plot t-SNE
sc.pl.tsne(short_read_shallow)

## Dimensionality Reduction: UMAP

In [ ]:
# Compute neighborhood graph
sc.pp.neighbors(short_read_shallow, n_neighbors=10, n_pcs=50)

# Run UMAP
sc.tl.umap(short_read_shallow)

# Plot UMAP
sc.pl.umap(short_read_shallow)

## Summary

This notebook performed comprehensive QC and exploratory analysis on the ENCODE short-read shallow C2C12 dataset:

1. **Data standardization**: Cleaned obs/var names
2. **QC filtering**: 
   - Removed low-quality cells (min_counts=1000, min_genes=750)
   - Removed low-detection genes (min_cells=2, min_counts=5)
   - Filtered high mitochondrial content cells (<20%)
3. **Normalization**: Library-size correction + log1p transformation
4. **Feature selection**: Identified highly variable genes
5. **Dimensionality reduction**: PCA, t-SNE, UMAP for visualization

The cleaned, normalized dataset is ready for downstream analysis including clustering and differential expression.